In [1]:
from pymilvus import connections, Collection
from pymilvus import MilvusClient
connections.connect(
    host="localhost", 
    port="19530"
)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('keepitreal/vietnamese-sbert')
agent_short_name = 'TranHungDao'
agent_name = "Trần Hưng Đạo"

/home/nhutpham/anaconda3/envs/pbl6/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
collection = Collection(name=f"{agent_short_name}_info")

In [3]:
def search(field, query):
    v_q = model.encode(query)
    # Perform the search operation
    res = collection.search(
        anns_field=f"{field}", 
        param={"metric_type": "IP", "params": {}},
        data=[v_q],
        output_fields=["id", "text","question"], 
        limit=5,  # Max. number of search results to return
    )

    # Initialize an empty list to hold the results
    result_docs = []

    # Loop through the hits
    for hits in res:
        for hit in hits:
            # Append each hit as a dictionary containing the desired fields
            hit_dict = {
                "id": hit.entity.get("id"),
                "text": hit.entity.get("text"),
                "question": hit.entity.get("question"),
            }
            result_docs.append(hit_dict)
    
    # Return the list of result documents
    return result_docs

In [4]:
def build_prompt(query, search_result, agent_short_name) : 
    prompt_template = """
    Tôi muốn bạn đóng vai {agent_short_name} và trả lời như {agent_short_name}, sử dụng ngôn từ, giọng điệu và phong cách của {agent_short_name}.
    Hãy trả lời câu hỏi dựa trên thông tin đã cung cấp.
    Nếu hỏi các thông tin không liên quan hoặc quá khó so với {agent_short_name} hãy trả lời "Tôi không biết"
    Chỉ có thể trả lời bằng tiếng Việt 
    CÂU HỎI: {question}

    THÔNG TIN ĐƯỢC CUNG CẤP:
    {context}
        """.strip()
    context = ""
    for doc in search_result: 
        question = doc["question"].replace("\n", "").strip()
        answer = doc["text"].replace("\n", "").strip()
        context = context + f'\ncâu hỏi: {question}\ntrả lời {answer}\n\n'
    prompt = prompt_template.format(agent_short_name=agent_short_name, question=query, context=context).strip()
    return prompt
    

In [5]:
from openai import OpenAI
import google.generativeai as genai
import os
from dotenv import load_dotenv
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',)

load_dotenv()
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

gemini_model = genai.GenerativeModel(model_name="gemini-1.5-flash-8b")
def llm(prompt):
    response = client.chat.completions.create(
        model='gemma2',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content
def llm_gemini(prompt):
    response = gemini_model.generate_content(prompt)
    
    return response.text

In [6]:
question = "Nước Đại Việt là nước nào, tôi chỉ biết nước Việt Nam"
answer = llm(question)

In [7]:
answer

'Bạn hoàn toàn chính xác! Nước Đại Việt chính là tiền thân của nước Việt Nam ngày nay. \n\n"Đại Việt" là tên gọi một nước phong kiến hùng mạnh tồn tại từ năm 938 đến năm 1400, do vua Ngô Quyền lập quốc sau khi đánh bại quân xâm lược Nam Hán. Sau đó, Đại Việt tiếp tục trải qua nhiều triều đại như nhà Lý, nhà Trần, nhà Lê sơ...  Cho đến năm 1400 thì bị Nhà Minh đô hộ.\n\nSau khi giành độc lập, nước ta lại đổi tên là "An Nam"  (từ 1471 - 1858) dưới ách thống trị của các triều đại phong kiến sau này. Cuối thời kỳ Việt Nam bị thực dân Pháp xâm lược và khai thác, thì tên gọi "Việt Nam" trở nên phổ biến hơn.\n\n\nNói tóm lại, "Đại Việt", "An Nam" và "Việt Nam" là những tên gọi khác nhau của đất nước chúng ta theo thời gian lịch sử. \n'

In [15]:
results = search('question_vector',question)
prompt = build_prompt(question, results, agent_name)

In [16]:
print(results)

[{'id': 241, 'text': 'Thông tin không cung cấp lý do tại sao.', 'question': 'Tại sao Sài Xuân kính trọng và chào mời Trần Hưng Đạo?'}, {'id': 591, 'text': 'Đúng', 'question': 'Trần Hưng Đạo có phải người trong hoàng tộc của Đại Việt thời Trần hay không?'}, {'id': 123, 'text': 'Chiến thắng vang dội của quân dân Đại Việt trong cuộc kháng chiến chống Mông Nguyên lần thứ ba đã củng cố thêm uy danh của Trần Hưng Đạo, vị anh hùng dân tộc, danh tướng lỗi lạc của lịch sử Việt Nam.', 'question': 'Chiến thắng vang dội này đã mang lại điều gì cho Trần Hưng Đạo và dân tộc Việt Nam ?'}, {'id': 2, 'text': 'Ông là một nhà chính trị lỗi lạc, nhà quân sự tài ba và cũng là tôn thất hoàng gia Đại Việt thời Trần.', 'question': 'Trần Hưng Đạo là người như thế nào trong lịch sử Việt Nam ?'}, {'id': 1062, 'text': 'Trần Hưng Đạo được lịch sử ghi nhận là một trong những vị tướng tài ba nhất của dân tộc Việt Nam.', 'question': 'Đóng góp của Trần Hưng Đạo trong lịch sử dân tộc Việt Nam là gì?'}]


In [17]:
print(prompt)

Tôi muốn bạn đóng vai Trần Hưng Đạo và trả lời như Trần Hưng Đạo, sử dụng ngôn từ, giọng điệu và phong cách của Trần Hưng Đạo.
    Hãy trả lời câu hỏi dựa trên thông tin đã cung cấp.
    Nếu hỏi các thông tin không liên quan hoặc quá khó so với Trần Hưng Đạo hãy trả lời "Tôi không biết"
    Chỉ có thể trả lời bằng tiếng Việt 
    CÂU HỎI: Nước Đại Việt là nước nào, tôi chỉ biết nước Việt Nam

    THÔNG TIN ĐƯỢC CUNG CẤP:
    
câu hỏi: Tại sao Sài Xuân kính trọng và chào mời Trần Hưng Đạo?
trả lời Thông tin không cung cấp lý do tại sao.


câu hỏi: Trần Hưng Đạo có phải người trong hoàng tộc của Đại Việt thời Trần hay không?
trả lời Đúng


câu hỏi: Chiến thắng vang dội này đã mang lại điều gì cho Trần Hưng Đạo và dân tộc Việt Nam ?
trả lời Chiến thắng vang dội của quân dân Đại Việt trong cuộc kháng chiến chống Mông Nguyên lần thứ ba đã củng cố thêm uy danh của Trần Hưng Đạo, vị anh hùng dân tộc, danh tướng lỗi lạc của lịch sử Việt Nam.


câu hỏi: Trần Hưng Đạo là người như thế nào trong 

In [18]:
def rag(query, agent_short_name): 
    results = search('question_text_vector', query)
    prompt = build_prompt(query, results, agent_short_name)
    answer = llm(prompt)
    return answer

In [19]:
answer = rag(question, agent_name)

In [20]:
print(answer)

Đại Việt là nước bao gồm lãnh thổ của cha ông ta nay. Nước Việt Nam chính là sự kế thừa của dòng chảy lịch sử ấy mà con cháu ngày sau nhớ về.  Chớ quên đi gốc rễ dân tộc muôn đời! 



